## **BAYESIAN MODELS**


**BASICS:**

Bayes Theorem:

![alt text](fig/bayes.png "Title")

Here, we replace H with hypothesis and D with data, Bayes' theorem tells us how to compute the probability of a hypothesis H given the data D. Where:

![alt text](fig/bayes_details.png "Title")

This means:

* **The prior distribution** should reflect what we know about the value of some parameter before seeing the data D.

* **The likelihood** is how we will introduce data in our analysis. It is an expression of the plausibility of the data given the parameters.

* **The posterior distribution** is the result of the Bayesian analysis and reflects all that we know about a problem (given our data and model). The posterior is a probability distribution for the parameters in our model and not a single value. **This distribution is a balance of the prior and the likelihood**. There is a joke that says: A Bayesian is one who, vaguely expecting a horse, and catching a glimpse of a donkey, strongly believes he has seen a mule. One way to kill the mood after hearing this joke is to explain that **if the likelihood and priors are both vague you will get a posterior reflecting vague beliefs about seeing a mule rather than strong ones**. 

We can think of the posterior as the updated prior in the light of the data. In fact, the posterior of one analysis can be used as the prior of a new analysis after collecting new data. This makes Bayesian analysis particularly suitable for analyzing data that becomes available in sequential order. 

The last term is the **evidence**, also known as marginal likelihood. Formally, the evidence is the probability of observing the data averaged over all the possible values the parameters can take. Most of the time we will not care about the evidence, and we will think of it as a simple normalization factor. This will not be problematic since we will only care about the relative values of the parameters and not their absolute ones:

![alt text](fig/bayes_alt.png "Title")

**Concepts to have in consideration:**

* **Conjugate prior** of a likelihood is a prior that, when used in combination with the given likelihood, returns a posterior with the same functional form as the prior. For example: every time we use a beta distribution as prior and a binomial distribution as likelihood, we will get a beta as a posterior. The Gaussian distribution is the conjugate prior of itself. 

**Results of a Bayesian Model**

* The result of a Bayesian analysis is the posterior distribution, not a single value but a distribution of plausible values given the data and our model.
* The most probable value is given by the mode of the posterior (the peak of the distribution).
* The spread of the posterior is proportional to the uncertainty about the value of a parameter; the more spread the distribution, the less certain we are.
* Given a sufficiently large amount of data, two or more Bayesian models with different priors will tend to converge to the same result. In the limit of infinite data, no matter which prior we use, we will always get the same posterior. Remember that infinite is a limit and not a number, so from a practical point of view in some cases the infinite amount of data could be approximated with a really small number of data points.